In [0]:
import glob
import os
import music21
import numpy as np
from music21 import converter, instrument, note, chord, duration, stream
import tensorflow as tf

print (music21.__version__) #if your version is lower than 4.x.x, you will encounter with some issues. 

In [0]:
# See figures inside jupyter notebook.
%matplotlib inline 

%config InlineBackend.figure_format = 'retina' # for Mac

In [0]:
def note_to_int(note): # converts the note's letter to pitch value which is integer form.
    # source: https://musescore.org/en/plugin-development/note-pitch-values
    # idea: https://github.com/bspaans/python-mingus/blob/master/mingus/core/notes.py
    
    note_base_name = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    if ('#-' in note):
        first_letter = note[0]
        base_value = note_base_name.index(first_letter)
        octave = note[3]
        value = base_value + 12*(int(octave)-(-1))
        
    elif ('#' in note): 
        first_letter = note[0]
        base_value = note_base_name.index(first_letter)
        octave = note[2]
        value = base_value + 12*(int(octave)-(-1))
        
    elif ('-' in note): 
        first_letter = note[0]
        base_value = note_base_name.index(first_letter)
        octave = note[2]
        value = base_value + 12*(int(octave)-(-1))
        
    else:
        first_letter = note[0]
        base_val = note_base_name.index(first_letter)
        octave = note[1]
        value = base_val + 12*(int(octave)-(-1))
        
    return value

In [0]:
# Lets determine our matrix's value 
# rest --> (min_value, lower_bound)
# continuation --> (lower_bound, upper_bound)
# first_touch --> (upper_bound, max_value)

min_value = 0.00
lower_first = 0.00

lower_second = 0.5
upper_first = 0.5

upper_second = 1.0
max_value = 1.0

def notes_to_matrix(notes, durations, offsets, min_value=min_value, lower_first=lower_first,
                    lower_second=lower_second,
                    upper_first=upper_first, upper_second=upper_second,
                    max_value=max_value):
    
    # I want to represent my notes in matrix form. X axis will represent time, Y axis will represent pitch values.
    # I should normalize my matrix between 0 and 1.
    # So that I will represent rest with (min_value, lower_first), continuation with [lower_second, upper_first]
    # and first touch with (upper_second, max_value)
    # First touch means that you press the note and it cause to 1 time duration playing. Continuation
    # represent the continuum of this note playing. 
    
    try:
        last_offset = int(offsets[-1]) 
    except IndexError:
        print ('Index Error')
        return (None, None, None)
    
    total_offset_axis = last_offset * 4 + (8 * 4) 
    our_matrix = np.random.uniform(min_value, lower_first, (128, int(total_offset_axis))) 
    # creates matrix and fills with (-1, -0.3), this values will represent the rest.
    
    for (note, duration, offset) in zip(notes, durations, offsets):
        how_many = int(float(duration)/0.25) # indicates time duration for single note.
       
        
        # Define difference between single and double note.
        # I have choose the value for first touch, the another value for continuation.
        # Lets make it randomize
        
        # I choose to use uniform distrubition. Maybe, you can use another distrubition like Gaussian.
        # I will try 
        first_touch = np.random.uniform(upper_second, max_value, 1)
        continuation = np.random.uniform(lower_second, upper_first, 1)
        
        if ('.' not in str(note)): # It is not chord. Single note.
            our_matrix[note, int(offset * 4)] = first_touch
            our_matrix[note, int((offset * 4) + 1) : int((offset * 4) + how_many)] = continuation

        else: # For chord
            chord_notes_str = [note for note in note.split('.')] 
            chord_notes_float = list(map(int, chord_notes_str)) # Take notes in chord one by one

            for chord_note_float in chord_notes_float:
                our_matrix[chord_note_float, int(offset * 4)] = first_touch
                our_matrix[chord_note_float, int((offset * 4) + 1) : int((offset * 4) + how_many)] = continuation
                
    return our_matrix

def check_float(duration): # This function fix the issue which comes from some note's duration. 
                           # For instance some note has duration like 14/3 or 7/3. 
    if ('/' in duration):
        numerator = float(duration.split('/')[0])
        denominator = float(duration.split('/')[1])
        duration = str(float(numerator/denominator))
    return duration

In [0]:
def midi_to_matrix(filename): # convert midi file to matrix for DL architecture.
  
    midi = music21.converter.parse(filename)
    notes_to_parse = None
    parts = music21.instrument.partitionByInstrument(midi)
    notes_to_parse = parts.parts[0].recurse()

    durations = []
    notes = []
    offsets = []
    
    for element in notes_to_parse:
        if isinstance(element, note.Note): # if it is single note
            notes.append(note_to_int(str(element.pitch)))
            duration = str(element.duration)[27:-1]
            durations.append(check_float(duration))
            offsets.append(element.offset)

        elif isinstance(element, chord.Chord): # if it is chord
            notes.append('.'.join(str(note_to_int(str(n)))
                                  for n in element.pitches))
            duration = str(element.duration)[27:-1]
            durations.append(check_float(duration))
            offsets.append(element.offset)

    
    
    our_matrix = notes_to_matrix(notes, durations, offsets)
    
    return our_matrix
    '''
    try:
        freq, time = our_matrix.shape
    except AttributeError:
        print ("'tuple' object has no attribute 'shape'")
        return None
            
    if (time >= length):
        return (our_matrix[:,:length]) # We have to set all individual note matrix to same shape for Generative DL.
    else:
        print ('%s have not enough duration' %(filename))
        '''

In [0]:
glob.glob("/content/drive/My Drive/Music/LSTM/*.h5")

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

glob.glob("/content/drive/My Drive/MusicBachViolin/*.mid")

In [0]:
a = 45
data=[]

for i, file in enumerate(glob.glob("/content/drive/My Drive/MusicBachViolin/*.mid")):
  data_temp = midi_to_matrix(file)
  for j in range(int(np.floor(data_temp[0,:].shape[0]/a))):
    data.append(data_temp[:,j:(j+a)])
  #data.append(data_temp)
    
midis_array = np.asarray(data)
    
midis_array = np.transpose(midis_array, (0, 2, 1)) 
midis_array = np.asarray(midis_array)
midis_array.shape   

In [0]:

midis_array = np.reshape(midis_array,(-1,128))
midis_array.shape

In [0]:
max_len = 15 # how many column will take account to predict next column.
step = 1 # step size.

previous_full = []
predicted_full = []

for i in range (0, midis_array.shape[0]-max_len, step):
    prev = midis_array[i:i+max_len,...] # take max_len column.
    pred = midis_array[i+max_len,...] # take (max_len)th column.
    previous_full.append(prev)
    predicted_full.append(pred)

In [0]:
previous_full = np.asarray(previous_full).astype('float64')
predicted_full = np.asarray (predicted_full).astype('float64')

In [0]:
num_of_sample, max_len, freq = previous_full.shape

print (previous_full.shape)
print (predicted_full.shape)

In [0]:
from keras import layers
from keras import models
import keras
from keras.models import Model
import tensorflow as tf
from keras.layers.advanced_activations import *


midi_shape = (max_len, 128)

# 这部分返回一个张量
input_midi = keras.Input(midi_shape)

# 层的实例是可调用的，它以张量为参数，并且返回一个张量
#x = layers.LSTM(512, return_sequences=True, unit_forget_bias=True)(input_midi)
x = layers.Bidirectional(layers.LSTM(512, return_sequences=True, unit_forget_bias=True))(input_midi)
#1024是输出维度，input_midi是输入值
x = layers.LeakyReLU()(x)
x = layers.BatchNormalization() (x)
#x = layers.Dropout(0.3)(x)

x = layers.LSTM(128, unit_forget_bias=True)(x)
x = layers.LeakyReLU()(x)
x = layers.BatchNormalization()(x)


x = layers.Dense(128, activation='softmax')(x) 

model = Model(input_midi, x)
# optimizer = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)
# optimizer = keras.optimizers.Nadam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-8, schedule_decay=0.004)
optimizer = keras.optimizers.SGD(lr=0.007)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [0]:
model.load_weights('/content/drive/My Drive/model3.h5')

In [0]:
midi_fulllist = []
for j in range(1):  
  start_index = random.randint(0, len(midis_array)- max_len - 1)
  midi_list = []
  for i in range(48):
      samples = midis_array[start_index+i: start_index+i+max_len]
      expanded_samples = np.expand_dims(samples, axis=0)
      preds = model.predict(expanded_samples, verbose=0)[0]
      preds = np.asarray(preds).astype('float64')
      midi_list.append(preds)    
  midi_fulllist.append(np.array(midi_list))

In [0]:

import matplotlib.pyplot as plt

import random
import sys

epoch_total = 10000
batch_size = 10

for epoch in range(0, 20): # Train model with epoch_total 
    print('Epoch:', epoch)
    
    #model.load_weights('my_model_weights.h5') # load model's weights.
    model.fit(previous_full, predicted_full, batch_size=batch_size, epochs=1,
              shuffle=True) # Fit model for 1 iteration.
    
   # start_index = random.randint(0, len(midis_array)- max_len - 1)
    
    #generated_midi = midis_array[start_index: start_index + max_len]
    #model.save_weights('my_model_weights.h5')
    model.save("my_model.h5")
    
    start_index = random.randint(0, len(midis_array)- max_len - 1)
    midi_list = []
    for i in range(30):
        samples = midis_array[start_index+i: start_index+i+max_len]
        expanded_samples = np.expand_dims(samples, axis=0)
        preds = model.predict(expanded_samples, verbose=0)[0]
        preds = np.asarray(preds).astype('float64')
        midi_list.append(preds)    
    implot = plt.imshow(np.array(midi_list), cmap="gray")
    plt.show()    
        

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Save Keras Model or weights on google drive

# create on Colab directory
model.save('model8.h5')    
model_file = drive.CreateFile({'title' : 'model8.h5'})
model_file.SetContentFile('model8.h5')
model_file.Upload()

# download to google drive
drive.CreateFile({'id': model_file.get('id')})

In [0]:
model.save_weights('model_weights7.h5')
weights_file = drive.CreateFile({'title' : 'model_weights7.h5'})
weights_file.SetContentFile('model_weights7.h5')
weights_file.Upload()
drive.CreateFile({'id': weights_file.get('id')})

In [0]:
# 3. reload weights from google drive into the model

# use (get shareable link) to get file id
last_weight_file = drive.CreateFile({'id': '1lVqs5_TLmPN5xPKg6Mgyw9ft_tEcAd51'}) 
last_weight_file.GetContentFile('last_weights.mat')
model.load_weights('last_weights.mat')

In [0]:
start_index = random.randint(0, len(midis_array)- max_len - 1)
samples = midis_array[start_index+i: start_index+i+max_len]
midi_list = []
for i in range(64):
    expanded_samples = np.expand_dims(samples, axis=0)
    preds = model.predict(expanded_samples, verbose=0)[0]
    preds = np.asarray(preds).astype('float64')
    samples = np.vstack([samples, preds])
    samples = np.round(samples[1:16,:]*2)/2
    midi_list.append(preds)
a = np.vstack([ midis_array[start_index+i: start_index+i+max_len], np.array(midi_list)])
implot = plt.imshow(a, cmap="gray")
plt.show()   

In [0]:
pip install midiutil

In [0]:
from google.colab import files
from midiutil.MidiFile import MIDIFile

matrix = np.asarray(a)
# create your MIDI object
mf = MIDIFile(1)     # only 1 track
track = 0   # the only track

time = 0    # start at the beginning
mf.addTrackName(track, time, "Sample Track")
mf.addTempo(track, time, 120)

# add some notes
channel = 0
volume = 100

for pitch in range(0,128):
  time = 0
  while time < 64:
    duration = 1
    if matrix[time, pitch]>=0.7:
      if time <63:
        while (matrix[time+duration, pitch]>=0.25)*(matrix[time+duration, pitch]<0.7)*(time+duration < 63):
            duration  = duration+1            
      mf.addNote(track, channel, pitch, time, duration, volume)
    if (matrix[time, pitch]>= 0.4)*(matrix[time-1, pitch]<0.25):  
      if time <63:
        while (matrix[time+duration, pitch]>=0.25)*(matrix[time+duration, pitch]<0.7)*(time+duration < 63):
            duration  = duration+1            
      mf.addNote(track, channel, pitch, time, duration, volume)      
    
    time +=1
    


# write it to disk
with open("output4.mid", 'wb') as outf:
    mf.writeFile(outf)  
    
for file in os.listdir(os.getcwd()):
    if file.endswith("output4.mid"):
      files.download(file) 